In [1]:
import os
import pandas as pd
import numpy as np
from haversine import haversine, Unit
import xgboost
import pickle as pk
from sklearn import preprocessing
from sklearn import pipeline
from sklearn import model_selection
from sklearn import linear_model
from sklearn import tree
from sklearn import svm
from sklearn import naive_bayes
from sklearn import neighbors
from sklearn import neural_network
from sklearn import ensemble
from sklearn import metrics

from data import feature_engineering

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(context='paper', font_scale=1.15)

data_dir = '../data/datathon_SC_ACN_22/'

c:\Users\linus\anaconda3\envs\datathon-fme2022\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


### Data

In [2]:
orders_data = pd.read_csv(data_dir + 'test.csv', delimiter=';', index_col='order_id')

cities_data = pd.read_csv(data_dir + 'cities_data.csv', delimiter=';')
product_data = pd.read_csv(data_dir + 'product_attributes.csv', delimiter=',', index_col='product_id')

In [3]:
orders_features = feature_engineering.engineer_features(orders_data, cities_data, product_data)

### Load model

In [4]:
with open('../models/ensemble_classifier_v001.pkl', 'rb') as f:
    model = pk.load(f)

In [10]:
predictions = model.predict_proba(orders_features)
predictions = [ probs[1] for probs in predictions ]

In [11]:
orders_data_predicted = orders_data.reset_index().join(pd.Series(predictions).rename('late_order')).set_index('product_id')

orders_data_predicted.sample(n=10)

,order_id,origin_port,3pl,customs_procedures,logistic_hub,customer,units,late_order
product_id,,,,,,,,
1651299,d922bb656961,Rotterdam,v_002,CRF,Lille,Prague,427,0.169229
1644062,68cdcbfa7ae3,Athens,v_002,DTD,Lille,Munich,492,0.311142
1683407,276eb0b8ec43,Athens,v_002,DTD,Venlo,Marseille,611,0.893256
1644308,386aba4c4d27,Rotterdam,v_002,CRF,Hamburg,Marseille,532,0.200078
1700738,4796a931c5a7,Rotterdam,v_004,DTD,Zaragoza,Madrid,560,0.403127
1687491,8a3327f4f643,Rotterdam,v_004,DTD,Hamburg,Milan,372,0.165656
1666519,1101a1688890,Athens,v_004,DTD,Venlo,Amsterdam,499,0.399231
1668606,997f13de1e26,Athens,v_004,DTD,Venlo,Copenhagen,498,0.456715
1680246,eea644eed81f,Rotterdam,v_004,DTD,Hamburg,Copenhagen,530,0.117572


In [13]:
orders_data_predicted[['order_id', 'late_order']].to_csv('../data/datathon_SC_ACN_22/test_predicted.csv', sep=',', index=False)